### WebCrawling
- 웹페이지의 종류
    - 정적페이지 : 페이지의 데이터가 변경될때 URL이 변경 O : HTML
    - 동적페이지 : 페이지의 데이터가 변경될때 URL이 변경 X : JSON
- requests package
    - 브라우져의 URL을 입력하면 서버에서 데이터를 다운받아 화면에 출력 : URL > DATA
    - requests 패키지 : URL > DATA

### Naver Stock Data
- Kospi 지수
- Kosdaq 지수
- USD : 원달러 환율

In [1]:
import requests
import pandas as pd

In [2]:
# 1. 웹서비스를 분석 : 크롬 개발자 도구 : URL
# https://m.stock.naver.com/

In [31]:
url = "https://m.stock.naver.com/api/index/KOSPI/price?pageSize=10&page=4"

In [32]:
# 2. request(url) > response(json) : JSON(str)

In [33]:
response = requests.get(url)
response

<Response [200]>

In [34]:
response.text[:200]

'[{"localTradedAt":"2022-06-22","closePrice":"2,342.81","compareToPreviousClosePrice":"-66.12","compareToPreviousPrice":{"code":"5","text":"하락","name":"FALLING"},"fluctuationsRatio":"-2.74","openPrice"'

In [35]:
# 3. JSON(str) > list, dict > DataFrame

In [36]:
data = response.json()
type(data), data[:1]

(list,
 [{'localTradedAt': '2022-06-22',
   'closePrice': '2,342.81',
   'compareToPreviousClosePrice': '-66.12',
   'compareToPreviousPrice': {'code': '5', 'text': '하락', 'name': 'FALLING'},
   'fluctuationsRatio': '-2.74',
   'openPrice': '2,417.11',
   'highPrice': '2,418.05',
   'lowPrice': '2,342.81'}])

In [15]:
df = pd.DataFrame(data)[["localTradedAt", "closePrice"]]
df.tail(2)

,localTradedAt,closePrice
8,2022-06-10,"2,595.87"
9,2022-06-09,"2,625.44"


In [16]:
# 4. 함수 만들기
# params : pagesize, page

In [17]:
def stock_price(pagesize, page):
    url = f"https://m.stock.naver.com/api/index/KOSPI/price?pageSize={pagesize}&page={page}"
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame(data)[["localTradedAt", "closePrice"]]

In [19]:
df = stock_price(30, 2)
df.tail(2)

,localTradedAt,closePrice
28,2022-05-11,"2,592.27"
29,2022-05-10,"2,596.56"


In [20]:
# KOSDAQ 데이터 수집 코드 작성

In [21]:
# 1. 웹서비스 분석 : URL

In [24]:
url = "https://m.stock.naver.com/api/index/KOSDAQ/price?pageSize=10&page=2"

In [22]:
# 2. request(url) > response(json) : JSON(str)

In [25]:
response = requests.get(url)
response

<Response [200]>

In [23]:
# 3. JSON(str) > list, dict > DataFrame

In [26]:
data = response.json()
df = pd.DataFrame(data)[["localTradedAt", "closePrice"]]
df.tail(2)

,localTradedAt,closePrice
8,2022-07-08,766.48
9,2022-07-07,757.97


In [44]:
def stock_price(pagesize, page, code="KOSPI"):
    """This function is crawling stock price from naver webpage.
    
    Params
    ------
    pagesize : int : one page size
    page : int : page number
    code : str : KOSPI or KOSDAQ
    
    Return
    ------
    type : DataFrame : display date, price columns
    """
    url = f"https://m.stock.naver.com/api/index/{code}/price?pageSize={pagesize}&page={page}"
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame(data)[["localTradedAt", "closePrice"]]

In [30]:
# stock_price(10, 1, "KOSDAQ")

In [39]:
kospi = stock_price(60, 1, "KOSPI")
kosdaq = stock_price(60, 1, "KOSDAQ")

In [45]:
# docstring : 함수를 사용하는 방법을 문자열로 작성
# help(), shift + tab
help(stock_price)

Help on function stock_price in module __main__:

stock_price(pagesize, page, code='KOSPI')
    This function is crawling stock price from naver webpage.
    
    Params
    ------
    pagesize : int : one page size
    page : int : page number
    code : str : KOSPI or KOSDAQ
    
    Return
    ------
    type : DataFrame : display date, price columns



#### 원달러 환율 데이터 수집
- https://m.stock.naver.com/ > 시장지표 > 미국 USD > 일별시세(더보기)

In [49]:
# 최근 60일치 원달러 환율 데이터 수집

In [50]:
# 1. 웹서비스 분석 : URL

In [54]:
page, pagesize = 1, 60
url = f"https://api.stock.naver.com/marketindex/exchange/FX_USDKRW/prices?page={page}&pageSize={pagesize}"
print(url)

https://api.stock.naver.com/marketindex/exchange/FX_USDKRW/prices?page=1&pageSize=60


In [51]:
# 2. request(url) > response(json) : JSON(str)

In [55]:
response = requests.get(url)

In [52]:
# 3. JSON(str) > list, dict > DataFrame

In [59]:
data = response.json()
usd = pd.DataFrame(data)[["localTradedAt", "closePrice"]]
usd.tail(2)

,localTradedAt,closePrice
58,2022-05-11,"1,274.50"
59,2022-05-10,"1,276.00"


In [60]:
kospi.tail(2)

,localTradedAt,closePrice
58,2022-05-11,"2,592.27"
59,2022-05-10,"2,596.56"


In [61]:
kosdaq.tail(2)

,localTradedAt,closePrice
58,2022-05-11,866.34
59,2022-05-10,856.14


In [ ]:
# 데이터분석
# 상관관계분석 : 두 데이터 집합 사이에 어떤 관계가 있는 확인하는 분석방법
# 원달러 환율이 높으면 코스피, 코스닥 지수가 낮다. > 음의 상관관계를 갖는다.

In [ ]:
# 피어슨 상관계수 : df.corr()
# 1과 가까울수록 강한 양의 상관관계를 갖는다.
# -1과 가까울수록 강한 음의 상관관계를 갖는다.
# 0과 가까울수록 관계가 없다.

In [106]:
# 데이터 전처리
df = kospi.copy()
df["kosdaq"] = kosdaq["closePrice"]
df["usd"] = usd["closePrice"]
df = df.rename(columns={"closePrice": "kospi"})
df.tail(2)

,localTradedAt,kospi,kosdaq,usd
58,2022-05-11,"2,592.27",866.34,"1,274.50"
59,2022-05-10,"2,596.56",856.14,"1,276.00"


In [107]:
df.dtypes

localTradedAt    object
kospi            object
kosdaq           object
usd              object
dtype: object

In [108]:
# 컬럼의 데이터 타입 변경 : str > float
# df[column].apply() : 모든 데이터를 함수에 대입한 결과를 출력
df["kospi"] = df["kospi"].apply(lambda data: float(data.replace(",", "")))
df["kosdaq"] = df["kosdaq"].apply(lambda data: float(data.replace(",", "")))
df["usd"] = df["usd"].apply(lambda data: float(data.replace(",", "")))

In [109]:
df.dtypes

localTradedAt     object
kospi            float64
kosdaq           float64
usd              float64
dtype: object

In [81]:
df[['kospi', 'kosdaq', 'usd']].corr()

,kospi,kosdaq,usd
kospi,1.000000,0.984055,-0.878234
kosdaq,0.984055,1.000000,-0.821246
usd,-0.878234,-0.821246,1.000000


In [ ]:
# kospi - kosdaq : 0.984 : 1과 가까우면 강한 양의 상관관계
# kospi - usd : -0.878 : -1과 가까우면 강한 음의 상관관계

In [ ]:
# copy(), apply, lambda

In [82]:
# copy()

In [87]:
data1 = [1, 2, 3]

data2 = data1        # 얕은복사(call by reference) : 주소값 복사
data3 = data1.copy() # 깊은복사(call by value) : 값 복사
print(data1, data2, data3) # [1, 2, 3], [1, 2, 3]

data1[1] = 4
print(data1, data2, data3) # [1, 4, 3], [1, 2, 3]

[1, 2, 3] [1, 2, 3] [1, 2, 3]
[1, 4, 3] [1, 4, 3] [1, 2, 3]


In [88]:
# apply(func) : 모든 데이터를 func을 적용시킨 결과 출력

In [91]:
df = pd.DataFrame([{"age": 23}, {"age": 36}, {"age": 27}])
df

,age
0,23
1,36
2,27


In [92]:
# 연령대 컬럼을 추가
def change_ages(age):
    return age // 10 * 10

In [95]:
df["age"]

0    23
1    36
2    27
Name: age, dtype: int64

In [98]:
df["ages"] = df["age"].apply(change_ages)
df

,age,ages
0,23,20
1,36,30
2,27,20


In [99]:
# lambda : 일회성 함수
# 사용이유 : 간단한 함수(파라미터를 받아서 바로 리턴하는)를 메모리를 절약하여 사용

In [105]:
# 함수 3개 사용, 메모리 3칸 사용
def plus(n1, n2):
    return n1 + n2

def minus(n1, n2):
    return n1 - n2

def calc(func, n1, n2):
    return func(n1, n2)

calc(plus, 1, 2), calc(minus, 1, 2)

(3, -1)

In [103]:
plus_lambda = lambda n1, n2: n1 + n2
plus(2, 3), plus_lambda(2, 3)

(5, 5)

In [104]:
# 함수 1개 사용, 메모리 1칸 사용
def calc(func, n1, n2):
    return func(n1, n2)

calc(lambda n1, n2: n1 + n2, 1, 2), calc(lambda n1, n2: n1 - n2, 1, 2)

(3, -1)

In [110]:
# summary

# 웹페이지의 종류
# - 정적페이지 : 데이터가 변경될때 URL 변경 O : HTML
# - 동적페이지 : 데이터가 변경될때 URL 변경 X : JSON

# 웹크롤링 절차
# 1. 웹서비스 분석 : 크롬 개발자 도구 : URL
# 2. request(url) > response(json) : JSON(str)
# 3. JSON(str) > list, dict > DataFrame